In [169]:
import pandas as pd
from bs4 import BeautifulSoup
import os
data_files = os.listdir('./data')

In [170]:
data_files = ['sedan.csv',
 'SupUV.csv',
 'wagon.csv',
 'coupe.csv',
 'minivan.csv',
 'pickups.csv',
 'hatchback.csv',
 'convertible.csv']

In [171]:
df = pd.DataFrame()
for data in data_files:
    file_path = './data/'+data
    d = pd.read_csv(file_path)
    d = d.T
    cols = d.iloc[0,:]
    d.columns = cols
    d = d.drop(['Unnamed: 0'],axis=0)
    d['type'] = data.replace('.csv','')
    df = pd.concat([df,d],axis=0,ignore_index=True)

In [172]:
import re
def get_model(text):
    

    match = re.search(r'([a-zA-Z0-9-]+)-\d{4}', text)
    model = '' 
    if match:
        model = match.group(1)
    # Replace hyphens with spaces
        model = model.replace("-", " ")
    return model
def get_year(text):
    year = ''
    match = re.search(r'-(2\d{3})-', text)

    if match:
        year = match.group(1)
    return year



In [173]:
df['make'] = df.source.map(lambda x: get_model(x).split(' ')[0])
df['year'] = df.source.map(lambda x: get_year(x))

In [174]:
df['model'] = df.source.map(lambda x: ' '.join(get_model(x).split(' ')[1:]))

In [175]:
def get_mileage(x):
    try:
        return float(x.split(' ')[0].replace(',',''))
    except:
        return ''

In [176]:
df['mileage_miles'] = df.miles.map(lambda x: get_mileage(x))

In [177]:
def extract(x):
    try:
        soup = BeautifulSoup(x, 'html.parser')


        details_rows = soup.find_all('div', class_='details-row')


        details_data = {}

        for row in details_rows:
            key = row.find('span', class_='details-property')
            value = row.find('span', class_='details-value')
            if key is not None and value is not None:
                key = key.get_text(strip=True)
                value = value.get_text(strip=True)
            details_data[key] = value
    except:
        pass
    
    return details_data

In [178]:
details = df.html.map(lambda x: extract(x))

In [179]:
details = pd.DataFrame(details.values.tolist())

In [180]:
df = pd.concat([df.drop(['html'],axis=1),details],axis=1)

In [181]:
def get_price(x):
    pattern = r'\$([\d,]+)'
    matches = re.findall(pattern, x)
    m = []
    try:
        for match in matches:
            match = match.replace(',', '')  # Remove commas
            m.append(float(match))
        return max(m)
    except:
        return 0

In [182]:
df['price'] = df.price.map(lambda x: get_price(x))

In [183]:
def features(x):
    
    feats = {}
    try:
        f = eval(x)
        for i in f:
            feats[i] = 1
    except:
        pass
    return feats

In [184]:
features = pd.DataFrame(df.features.map(lambda x: features(x)).values.tolist())

In [185]:
df = pd.concat([df,features],axis=1)

In [186]:
df = df.drop([ 'miles','source','features'],axis=1)

In [197]:
df = df.drop([None],axis=1)

In [198]:
df

,price,type,make,year,model,mileage_miles,Exterior Color,Interior Color,Body Style,Doors,...,Lane Assist,Parking Assist System,Stability Control,Adaptive Cruise Control,Alloy Wheels,Cooled Seats,Full Self-Driving Capability,Third Row Seating,Tow Hitch / Package,Rear Seat Entertainment
0,32589.0,sedan,subaru,2022,wrx,14172.0,White,Black,Sedan,4 Doors,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,32589.0,sedan,audi,2022,a3,7729.0,Silver,Black,Sedan,4 Doors,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,48589.0,sedan,bmw,2023,5 series,25001.0,Blue,Brown,Sedan,4 Doors,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,24589.0,sedan,hyundai,2023,elantra,8677.0,Red,Gray,Sedan,4 Doors,...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,23999.0,sedan,hyundai,2023,elantra,16506.0,Blue,Black,Sedan,4 Doors,...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2892,63999.0,convertible,mercedes,2019,benz e class,16461.0,Black,Black,Convertible,2 Doors,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2893,26589.0,convertible,mercedes,2014,benz e class,44455.0,Silver,Black,Convertible,2 Doors,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2894,22999.0,convertible,fiat,2017,124 spider,38882.0,Gray,Black,Convertible,2 Doors,...,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2895,28999.0,convertible,ford,2014,mustang,23049.0,Gray,Black,Convertible,2 Doors,...,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [207]:
df.columns

Index(['price', 'type', 'make', 'year', 'model', 'mileage_miles',
       'Exterior Color', 'Interior Color', 'Body Style', 'Doors', 'MPG', 'VIN',
       'Engine', 'Transmission', 'Drive Type', 'Fuel', 'Tank Size',
       'Bed Style', 'Cab Style', 'Android Auto', 'Apple CarPlay',
       'Backup Camera / Assist', 'Bluetooth', 'Heated Seats',
       'Hill Assist System', 'Keyless Entry', 'Keyless Ignition',
       'Multimedia / Telematics', 'Premium Sound System', 'Satellite Radio',
       'Sunroof / Moonroof', 'Leather Seats', 'Power Seats',
       'Traction Control', 'Driver Assistance / Confidence Pkg',
       'Head-Up Display', 'Lane Departure Warning', 'Navigation System',
       'Remote Start', 'Blind Spot Monitor', 'Lane Assist',
       'Parking Assist System', 'Stability Control', 'Adaptive Cruise Control',
       'Alloy Wheels', 'Cooled Seats', 'Full Self-Driving Capability',
       'Third Row Seating', 'Tow Hitch / Package', 'Rear Seat Entertainment'],
      dtype='object')

In [203]:
df.shape

(2897, 50)

In [209]:
df.to_excel('./data/master_data.xlsx')